In [1]:
import pandas as pd

df = pd.read_csv('redditdataset.txt', sep='\t', lineterminator='\n',header = None)
size = len(df)

In [2]:
df.columns = ['v1', 'v2', 'timestamp']

In [3]:
df1 = df.head(3*size // 4)

In [4]:
df2 = df.tail(1*size // 4 + 1)

In [5]:
from collections import defaultdict
all_connections = defaultdict(list)
for index, row in df1.iterrows():
        all_connections[row["v1"]].append((row["v2"], row["timestamp"]))

In [6]:
all_connections_test = defaultdict(list)
for index, row in df2.iterrows():
    if row["v1"] not in all_connections:
        all_connections_test[row["v1"]].append((row["v2"], row["timestamp"]))
    else:
        all_connections[row["v1"]].append((row["v2"], row["timestamp"]))

In [10]:
cold_dict = defaultdict(list)
for index, row in df1.iterrows():
    cold_dict[row["v2"]].append((row["v1"], row["timestamp"]))

In [9]:
from random import sample
j_hyp = 0.0
j_rand = 0.0
conn_avg = 0.0
# key is the cold start node in question
for key in all_connections_test:
    try:
        #val is the first node it connects with (the list in all_connections_test is sorted by timestamp)
        val = all_connections_test[key][0][0]
    except:
        continue
    #the list of nodes key actually connects with after cold starting
    ground_truth = [elem[0] for elem in all_connections_test[key]]
    try:
        # for the last thing that cold started with val, what else did it connect with?
        hyp = [elem[0] for elem in all_connections[all_connections[val][-1][0]]]
    except:
        continue
    #print(ground_truth , hyp)
    l = lambda x, y : x if x < y else y
    conn_avg += (len(hyp) / len(all_connections[val]))
    rand = sample([elem[0] for elem in all_connections[val]], l(len(hyp),len(all_connections[val])))
    j_hyp+= jaccard_similarity(ground_truth, hyp)
    j_rand+= jaccard_similarity(ground_truth, rand)
    
print(j_hyp)
print(j_rand)
print(conn_avg / len(all_connections_test.keys()))

162.39753671499892
25.911766419557374
11.47155320625007


In [44]:
set(all_connections_test.keys()).intersection(set(all_connections.keys()))

set()

In [8]:
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(set(list2))))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection / union)